<a href="https://colab.research.google.com/github/emanuel711/Big-Data/blob/main/bigdata_books_rating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Big data Books Rating**

In [ ]:
!apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null  
!wget -q https://archive.apache.org/dist/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz  
!tar xf spark-3.3.1-bin-hadoop3.tgz  
!pip install -q findspark 

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 3s (103 kB/s)
Reading package lis

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

In [ ]:
import findspark
findspark.init("spark-3.3.1-bin-hadoop3") 
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark.sql import Row
from pyspark.sql import functions

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/MyDrive/Big Data Project/ProjectBigDataBooksRating"  

'bigdata books rating.ipynb'	       BX-Books.csv
'Big Data Project-Books Rating.pptx'   BX-Users.csv
'Book1 (1).twb'			       lightbulb_insight.png
'Book Rating.twb'		       rating.csv
 books.csv			       users.csv
 BX-Book-Ratings.csv


**Importing files**

In [ ]:
dfb = spark.read.option("delimiter", ";").option("header", True).option("mode","DROPMALFORMED").csv("/content/drive/MyDrive/Big Data Project/ProjectBigDataBooksRating/BX-Books.csv")
dfu = spark.read.option("delimiter", ";").option("header", True).option("mode","DROPMALFORMED").csv("/content/drive/MyDrive/Big Data Project/ProjectBigDataBooksRating/BX-Users.csv")
dfr = spark.read.option("delimiter", ";").option("header", True).option("mode","DROPMALFORMED").csv("/content/drive/MyDrive/Big Data Project/ProjectBigDataBooksRating/BX-Book-Ratings.csv")


**dfr file ETA fact table**

In [ ]:
dfr.show()

+--------------------+------------------+
|   "User-ID;""ISBN""|""Book-Rating""",,|
+--------------------+------------------+
|"276725;""0345451...|          ""0""",,|
|"276726;""0155061...|          ""5""",,|
|"276727;""0446520...|          ""0""",,|
|"276729;""0521656...|          ""3""",,|
|"276729;""0521795...|          ""6""",,|
|"276733;""2080674...|          ""0""",,|
|"276736;""3257224...|          ""8""",,|
|"276737;""0600570...|          ""6""",,|
|"276744;""0385501...|          ""7""",,|
|"276745;""3423105...|         ""10""",,|
|"276746;""0425115...|          ""0""",,|
|"276746;""0449006...|          ""0""",,|
|"276746;""0553561...|          ""0""",,|
|"276746;""0553564...|          ""0""",,|
|"276746;""0786013...|          ""0""",,|
|"276746;""0786014...|          ""0""",,|
|"276747;""0060517...|          ""9""",,|
|"276747;""0451192...|          ""0""",,|
|"276747;""0609801...|          ""0""",,|
|"276747;""0671537...|          ""9""",,|
+--------------------+------------

In [ ]:
print((dfr.count(), len(dfr.columns)))

(1048575, 2)


In [ ]:
dfr=dfr.withColumnRenamed('"User-ID;""ISBN""', 'UserID-ISBN')
dfr=dfr.withColumnRenamed('""Book-Rating""",,','bookRating')

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType
dfr=dfr.withColumn('bookRating',regexp_replace('bookRating',',',''))
dfr=dfr.withColumn('bookRating',regexp_replace('bookRating','"',''))
dfr=dfr.withColumn('UserID-ISBN',regexp_replace('UserID-ISBN','"',''))
dfr = dfr.withColumn('UserID', split(dfr['UserID-ISBN'], ';').getItem(0)) \
       .withColumn('ISBN', split(dfr['UserID-ISBN'], ';').getItem(1))
dfr=dfr.drop("UserID-ISBN") 
dfr = dfr.withColumn("userID", dfr["userID"].cast(IntegerType()))
dfr=dfr.sort("userID")      
dfr = dfr.select('UserID',"ISBN","bookrating")


In [ ]:
dfr=dfr.sort("userID")
x = dfr.withColumn("UserID",dfr.UserID.cast('int'))
dfr=x
dfr=dfr.withColumn("bookrating",dfr.bookrating.cast('int'))
dfr.show()

+------+----------+----------+
|UserID|      ISBN|bookrating|
+------+----------+----------+
|     2|0195153448|         0|
|     7| 034542252|         0|
|     8|1881320189|         7|
|     8|1575663937|         6|
|     8|0399135782|         0|
|     8|074322678X|         5|
|     8|0887841740|         5|
|     8|1567407781|         6|
|     8|0393045218|         0|
|     8|0771025661|         0|
|     8|0671870432|         0|
|     8|0374157065|         0|
|     8|0771074670|         0|
|     8|080652121X|         0|
|     8|1558746218|         0|
|     8|0002005018|         5|
|     8|0425176428|         0|
|     8|0060973129|         0|
|     8|1552041778|         5|
|     8|0679425608|         0|
+------+----------+----------+
only showing top 20 rows



In [ ]:
dfr.printSchema()

root
 |-- UserID: integer (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- bookrating: integer (nullable = true)



In [ ]:
print((dfr.count(), len(dfr.columns)))

(1048575, 3)


**dfu file ETA dim table**

In [ ]:
dfu.show()

+-------+--------------------+----+
|User-ID|            Location| Age|
+-------+--------------------+----+
|      1|  nyc, new york, usa|NULL|
|      2|stockton, califor...|  18|
|      3|moscow, yukon ter...|NULL|
|      4|porto, v.n.gaia, ...|  17|
|      5|farnborough, hant...|NULL|
|      6|santa monica, cal...|  61|
|      7| washington, dc, usa|NULL|
|      8|timmins, ontario,...|NULL|
|      9|germantown, tenne...|NULL|
|     10|albacete, wiscons...|  26|
|     11|melbourne, victor...|  14|
|     12|fort bragg, calif...|NULL|
|     13|barcelona, barcel...|  26|
|     14|mediapolis, iowa,...|NULL|
|     15|calgary, alberta,...|NULL|
|     16|albuquerque, new ...|NULL|
|     17|chesapeake, virgi...|NULL|
|     18|rio de janeiro, r...|  25|
|     19|           weston, ,|  14|
|     20|langhorne, pennsy...|  19|
+-------+--------------------+----+
only showing top 20 rows



In [ ]:
print((dfu.count(), len(dfu.columns)))

(278859, 3)


In [ ]:
x = dfu.withColumn("Age",dfu.Age.cast('int'))
dfu = x.fillna(-1)
dfu = dfu.withColumn('city', split(dfu['Location'], ',').getItem(0)) \
       .withColumn('state', split(dfu['Location'], ',').getItem(1)).withColumn('country', split(dfu['Location'], ',').getItem(2))
dfu=dfu.drop('Location')
dfu=dfu.withColumnRenamed("User-ID", "UserID")
dfu = dfu.withColumn('city', when(dfu.city.contains('�'),'unknown') \
      .otherwise(dfu.city))
dfu = dfu.withColumn('state', when(dfu.state.contains('�'),'unknown') \
      .otherwise(dfu.state))
dfu = dfu.withColumn('country', when(dfu.country.contains('�'),'unknown') \
      .otherwise(dfu.country))

dfu.show()

+------+---+--------------+----------------+---------------+
|UserID|Age|          city|           state|        country|
+------+---+--------------+----------------+---------------+
|     1| -1|           nyc|        new york|            usa|
|     2| 18|      stockton|      california|            usa|
|     3| -1|        moscow| yukon territory|         russia|
|     4| 17|         porto|        v.n.gaia|       portugal|
|     5| -1|   farnborough|           hants| united kingdom|
|     6| 61|  santa monica|      california|            usa|
|     7| -1|    washington|              dc|            usa|
|     8| -1|       timmins|         ontario|         canada|
|     9| -1|    germantown|       tennessee|            usa|
|    10| 26|      albacete|       wisconsin|          spain|
|    11| 14|     melbourne|        victoria|      australia|
|    12| -1|    fort bragg|      california|            usa|
|    13| 26|     barcelona|       barcelona|          spain|
|    14| -1|    mediapol

In [ ]:
for i in dfu.columns:
  dfu=dfu.withColumn(i,regexp_replace(i,'n/a','unknown'))
  dfu=dfu.withColumn(i,regexp_replace(i,'"\"',''))
  dfu=dfu.withColumn(i,regexp_replace(i,'"?"',''))
  dfu=dfu.withColumn(i,regexp_replace(i,'"',''))
  dfu=dfu.withColumn(i,regexp_replace(i,'_',''))
  dfu=dfu.withColumn(i,regexp_replace(i,'"*"','unknown'))
  dfu=dfu.withColumn(i,regexp_replace(i,'#','unknown'))

In [ ]:
from pyspark.sql.functions import col,when
dfu1=dfu.select([when(col(c)=="",'unknown').otherwise(col(c)).alias(c) for c in dfu.columns])
dfu=dfu1
dfu1=dfu.select([when(col(c)==" ",'unknown').otherwise(col(c)).alias(c) for c in dfu.columns])
dfu=dfu1
dfu.show()

+------+---+--------------+----------------+---------------+
|UserID|Age|          city|           state|        country|
+------+---+--------------+----------------+---------------+
|     1| -1|           nyc|        new york|            usa|
|     2| 18|      stockton|      california|            usa|
|     3| -1|        moscow| yukon territory|         russia|
|     4| 17|         porto|        v.n.gaia|       portugal|
|     5| -1|   farnborough|           hants| united kingdom|
|     6| 61|  santa monica|      california|            usa|
|     7| -1|    washington|              dc|            usa|
|     8| -1|       timmins|         ontario|         canada|
|     9| -1|    germantown|       tennessee|            usa|
|    10| 26|      albacete|       wisconsin|          spain|
|    11| 14|     melbourne|        victoria|      australia|
|    12| -1|    fort bragg|      california|            usa|
|    13| 26|     barcelona|       barcelona|          spain|
|    14| -1|    mediapol

In [ ]:
from pyspark.sql.functions import when
dfu = dfu.withColumn("Age", when(dfu.Age > 100,-1) \
      .when(dfu.Age <10,-1) \
      .otherwise(dfu.Age))

In [ ]:
dfu.show()

+------+---+--------------+----------------+---------------+
|UserID|Age|          city|           state|        country|
+------+---+--------------+----------------+---------------+
|     1| -1|           nyc|        new york|            usa|
|     2| 18|      stockton|      california|            usa|
|     3| -1|        moscow| yukon territory|         russia|
|     4| 17|         porto|        v.n.gaia|       portugal|
|     5| -1|   farnborough|           hants| united kingdom|
|     6| 61|  santa monica|      california|            usa|
|     7| -1|    washington|              dc|            usa|
|     8| -1|       timmins|         ontario|         canada|
|     9| -1|    germantown|       tennessee|            usa|
|    10| 26|      albacete|       wisconsin|          spain|
|    11| 14|     melbourne|        victoria|      australia|
|    12| -1|    fort bragg|      california|            usa|
|    13| 26|     barcelona|       barcelona|          spain|
|    14| -1|    mediapol

In [ ]:
dfu.dtypes

[('UserID', 'string'),
 ('Age', 'string'),
 ('city', 'string'),
 ('state', 'string'),
 ('country', 'string')]

In [ ]:
dfu=dfu.withColumn("UserID",dfu.UserID.cast('int'))
dfu=dfu.withColumn("Age",dfu.Age.cast('int'))
dfu.dtypes

[('UserID', 'int'),
 ('Age', 'int'),
 ('city', 'string'),
 ('state', 'string'),
 ('country', 'string')]

In [ ]:
print((dfu.count(), len(dfu.columns)))

(278859, 5)


In [ ]:
dfb.show()

+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|      ISBN|          Book-Title|         Book-Author|Year-Of-Publication|           Publisher|         Image-URL-S|         Image-URL-M|         Image-URL-L|
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|               2002|Oxford University...|http://images.ama...|http://images.ama...|http://images.ama...|
|0002005018|        Clara Callan|Richard Bruce Wright|               2001|HarperFlamingo Ca...|http://images.ama...|http://images.ama...|http://images.ama...|
|0060973129|Decision in Normandy|        Carlo D'Este|               1991|     HarperPerennial|http://images.ama...|http://images.ama...|http://images.ama...|
|0374157065|Flu: The Story of...|    Gina Bari

In [ ]:
print((dfb.count(), len(dfb.columns)))

(271379, 8)


**dfb file ETA dim table**

In [ ]:
y=dfb.drop('Image-URL-S', 'Image-URL-M','Image-URL-L')
dfb=y
dfb=dfb.withColumnRenamed('Year-Of-Publication', 'PublishY')
dfb=dfb.withColumnRenamed('Book-Title','Title')
dfb=dfb.withColumnRenamed('Book-Author' ,'Author')


In [ ]:
for i in dfb.columns:
  dfb=dfb.withColumn(i,regexp_replace(i,'�?',''))
  dfb=dfb.withColumn(i,regexp_replace(i,'¼',''))

dfb=dfb.withColumn('Author',regexp_replace('Author','"\"',''))
dfb=dfb.withColumn('Author',regexp_replace('Author','"',''))

dfb.show()

+----------+--------------------+--------------------+--------+--------------------+
|      ISBN|               Title|              Author|PublishY|           Publisher|
+----------+--------------------+--------------------+--------+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|    2002|Oxford University...|
|0002005018|        Clara Callan|Richard Bruce Wright|    2001|HarperFlamingo Ca...|
|0060973129|Decision in Normandy|        Carlo D'Este|    1991|     HarperPerennial|
|0374157065|Flu: The Story of...|    Gina Bari Kolata|    1999|Farrar Straus Giroux|
|0393045218|The Mummies of Ur...|     E. J. W. Barber|    1999|W. W. Norton &amp...|
|0399135782|The Kitchen God's...|             Amy Tan|    1991|    Putnam Pub Group|
|0425176428|What If?: The Wor...|       Robert Cowley|    2000|Berkley Publishin...|
|0671870432|     PLEADING GUILTY|         Scott Turow|    1993|          Audioworks|
|0679425608|Under the Black F...|     David Cordingly|    1996|  

In [ ]:
dfb.filter(dfb.Author=='Lulu').collect()

[Row(ISBN='0751533718', Title='Lulu', Author='Lulu', PublishY='2003', Publisher='Time Warner Paperbacks')]

In [ ]:
dfb=dfb.withColumn("PublishY",dfb.PublishY.cast('int'))
dfb = dfb.withColumn("PublishY", when(dfb.PublishY > 2022 ,0).otherwise(dfb.PublishY))

In [ ]:
dfb.filter("PublishY=0").show()

+----------+--------------------+--------------------+--------+--------------------+
|      ISBN|               Title|              Author|PublishY|           Publisher|
+----------+--------------------+--------------------+--------+--------------------+
|3150000335|    Kabale Und Liebe|            Schiller|       0|Philipp Reclam, J...|
|342311360X|Die Liebe in Den ...|Gabriel Garcia Ma...|       0|Deutscher Taschen...|
|0571197639|Poisonwood Bible ...|  Barbara Kingsolver|       0|     Faber Faber Inc|
|3596214629|Herr Der Fliegen ...|             Golding|       0|Fischer Taschenbu...|
|8845229041|Biblioteca Univer...|            P Coelho|       0|  Fabbri - RCS Libri|
|0345251733|Rainbow the Storm...|   Christopher Finch|       0|    Ballantine Books|
|0590021117|Little House On t...| Laura Ingall Wilder|       0|      Scholastic Inc|
|2070362388|              Ravage|       Rene Barjavel|       0|    Gallimard French|
|2266033689|  Le Diable Au Corps|    Raymond Radiguet|       0|Pr

In [ ]:
print((dfb.count(), len(dfb.columns)))

(271379, 5)


In [ ]:
import pyspark.sql.functions as fn
from pyspark.sql import Window
type(dfb)

pyspark.sql.dataframe.DataFrame

In [ ]:
dfu.coalesce(1).write.option("header",True).csv("/content/drive/MyDrive/Big Data Project/ProjectBigDataBooksRating/Users.csv")

In [ ]:
 dfb.coalesce(1).write.option("header",True).csv("/content/drive/MyDrive/Big Data Project/ProjectBigDataBooksRating/Books.csv")

In [ ]:
dfr.coalesce(1).write.option("header",True).csv("/content/drive/MyDrive/Big Data Project/ProjectBigDataBooksRating/Rating.csv")

In [ ]:
spark.stop()
